In [ ]:
import cv2
import torch
import math
import numpy as np
import pandas as pd
from typing import Tuple, Optional, List
from ultralytics import YOLO

class SyringeVolumeCalculator:
    def __init__(self) -> None:
        self.model = YOLO("runs/pose/train-pose11x-v20/weights/best.pt").eval()
        # Choose device: cuda > mps > cpu
        if torch.cuda.is_available():
            self.device: str = "cuda"
        elif torch.backends.mps.is_available():
            self.device = "mps"
        else:
            self.device = "cpu"
        self.model.to(self.device)
        self.syringe_diameter: float = 1.0  # cm

    @staticmethod
    def calculate_line_equation(p1: np.ndarray, p2: np.ndarray) -> Tuple[float, float, float]:
        x1, y1 = p1
        x2, y2 = p2
        A: float = y2 - y1
        B: float = x1 - x2
        C: float = (x2 - x1) * y1 - (y2 - y1) * x1
        return (A, B, C)

    def calculate_parallel_distance(
        self,
        line1: Tuple[float, float, float],
        line2: Tuple[float, float, float]
    ) -> Optional[float]:
        A1, B1, C1 = line1
        A2, B2, C2 = line2
        if abs(A1 * B2 - A2 * B1) > 1e-6:
            return None
        return abs(C2 - C1) / math.hypot(A1, B1)

    @staticmethod
    def point_to_line_distance(point: np.ndarray, line: Tuple[float, float, float]) -> float:
        x, y = point
        A, B, C = line
        return abs(A * x + B * y + C) / math.hypot(A, B)

    def calculate_height_for_detection(
        self,
        ll_point: np.ndarray,
        ul_point: np.ndarray,
        ur_point: np.ndarray,
        lr_point: np.ndarray,
        scale_factor: float
    ) -> float:
        lower_line_eq: Tuple[float, float, float] = self.calculate_line_equation(ll_point, lr_point)
        upper_line_eq: Tuple[float, float, float] = self.calculate_line_equation(ul_point, ur_point)
        h_pixels: Optional[float] = self.calculate_parallel_distance(lower_line_eq, upper_line_eq)
        d_ll: float = self.point_to_line_distance(ll_point, upper_line_eq)
        d_lr: float = self.point_to_line_distance(lr_point, upper_line_eq)
        avg_point_dist: float = (d_ll + d_lr) / 2
        if h_pixels is None or abs(h_pixels - avg_point_dist) > 5:
            return avg_point_dist * scale_factor
        return ((h_pixels * 0.7) + (avg_point_dist * 0.3)) * scale_factor

    def process_frame(self, frame: np.ndarray) -> Tuple[np.ndarray, List[float]]:
        # Run inference and get an annotated frame (with on-screen pose dots).
        results = self.model.predict(frame, verbose=False, conf=0.6)
        if not results:
            return frame, []
        result = results[0]
        annotated_frame: np.ndarray = result.plot()  # Draws the default keypoints and confidence dots.
        volumes: List[float] = []
        try:
            # Filter out any missing keypoints.
            keypoints_list = [kp for kp in result.keypoints.xy if kp is not None]
            # Get confidence values if available.
            if hasattr(result.keypoints, "conf") and result.keypoints.conf is not None:
                conf_numpy = result.keypoints.conf.cpu().numpy()
                confidences = list(conf_numpy)
            else:
                confidences = [None] * len(keypoints_list)
        except Exception as e:
            print(f"Error processing keypoints: {e}")
            return annotated_frame, volumes

        for i, kp in enumerate(keypoints_list):
            if kp is None:
                continue
            try:
                # Convert to numpy array if necessary.
                kpts: np.ndarray = kp.cpu().numpy() if hasattr(kp, "cpu") else kp
            except Exception as e:
                print(f"Error converting keypoints to numpy: {e}")
                continue

            if kpts.shape[0] < 4:
                continue

            # Retrieve confidence and convert it to a float if needed.
            conf = confidences[i] if i < len(confidences) else None
            ul_point: np.ndarray = kpts[1]
            if conf is not None:
                try:
                    conf_val = conf.item() if hasattr(conf, "item") else float(conf)
                    cv2.putText(annotated_frame, f"Conf: {conf_val:.2f}",
                                (int(ul_point[0]), int(ul_point[1]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                except Exception as e:
                    print(f"Error formatting confidence: {e}")

            lower_edge_px: float = np.linalg.norm(kpts[3] - kpts[0])
            upper_edge_px: float = np.linalg.norm(kpts[2] - kpts[1])
            if lower_edge_px <= 0 or upper_edge_px <= 0:
                continue
            scale_factor: float = 1.0 / ((lower_edge_px + upper_edge_px) / 2.0)
            height_cm: float = self.calculate_height_for_detection(kpts[0], kpts[1], kpts[2], kpts[3], scale_factor)
            if height_cm <= 0 or height_cm > 30:
                continue
            volume: float = math.pi * (self.syringe_diameter / 2) ** 2 * height_cm
            volumes.append(volume)
            # Overlay volume and height text.
            cv2.putText(annotated_frame, f"Volume {i+1}: {volume:.2f} mL", (10, 30 + i * 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"Height {i+1}: {height_cm:.1f} cm", (10, 70 + i * 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        return annotated_frame, volumes

    def run_on_video(
        self,
        input_video_path: str,
        output_video_path: str,
        output_csv_path: str
    ) -> None:
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            raise IOError(f"Cannot open video file: {input_video_path}")
        width: int = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height: int = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps: float = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:
            fps = 30
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        frame_count: int = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        current_frame: int = 0
        csv_rows: List[dict] = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            processed_frame, volumes = self.process_frame(frame)
            out.write(processed_frame)
            row: dict = {"frame": current_frame}
            for idx, vol in enumerate(volumes):
                row[f"volume_estimation{idx+1}"] = vol
            csv_rows.append(row)
            current_frame += 1
            print(f"Processing frame {current_frame}/{frame_count}", end="\r")
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("\nExport complete! 🎉")
        df = pd.DataFrame(csv_rows)
        df.to_csv(output_csv_path, index=False)
        print(f"CSV data exported to {output_csv_path}.")

# Set your file paths accordingly.
input_video_path: str = "../videos/input_videos/IMG_4594.MOV"     # Replace with the input video file path.
output_video_path: str = "../videos/output_videos/oxoxxx.mp4"   # Replace with the desired output video file path.
output_csv_path: str = "volume_estimations.csv"  # Replace with the desired CSV file path.

# Create an instance and run the video processing.
calculator = SyringeVolumeCalculator()
calculator.run_on_video(input_video_path, output_video_path, output_csv_path)

../videos/input_videos/IMG_4593.MOV

In [ ]:
import cv2
import torch
import math
import numpy as np
from typing import Tuple, Optional
from ultralytics import YOLO

class SyringeVolumeCalculator:
    def __init__(self) -> None:
        # Initialize YOLO model and set device
        self.model = YOLO("runs/pose/train-pose11x-v23-P50/weights/best.pt").eval()
        if torch.cuda.is_available():
            self.device = "cuda"
        elif torch.backends.mps.is_available():
            self.device = "mps"
        else:
            self.device = "cpu"
        self.model.to(self.device)
        # List of possible diameters in cm
        self.possible_diameters = [0.45, 1.0, 1.25, 2.0]
        # Keypoint variables
        self.ll_point: Optional[np.ndarray] = None
        self.ul_point: Optional[np.ndarray] = None
        self.ur_point: Optional[np.ndarray] = None
        self.lr_point: Optional[np.ndarray] = None

    @staticmethod
    def calculate_line_equation(p1: np.ndarray, p2: np.ndarray) -> Tuple[float, float, float]:
        """Calculate line equation coefficients A, B, C from two points."""
        x1, y1 = p1
        x2, y2 = p2
        A = y2 - y1
        B = x1 - x2
        C = (x2 - x1) * y1 - (y2 - y1) * x1
        return (A, B, C)

    def calculate_parallel_distance(self, line1: Tuple[float, float, float],
                                    line2: Tuple[float, float, float]) -> Optional[float]:
        """Calculate distance between two parallel lines, return None if not parallel."""
        A1, B1, C1 = line1
        A2, B2, C2 = line2
        if abs(A1 * B2 - A2 * B1) > 1e-6:  # Not parallel
            return None
        return abs(C2 - C1) / math.hypot(A1, B1)

    @staticmethod
    def point_to_line_distance(point: np.ndarray, line: Tuple[float, float, float]) -> float:
        """Calculate perpendicular distance from a point to a line."""
        x, y = point
        A, B, C = line
        return abs(A * x + B * y + C) / math.hypot(A, B)

    def draw_debug_info(self, frame: np.ndarray) -> None:
        """Draw debug information like keypoint labels and edge lengths."""
        debug_params = {
            'fontFace': cv2.FONT_HERSHEY_SIMPLEX,
            'fontScale': 0.6,
            'color': (0, 0, 255),
            'thickness': 1
        }
        labels = ['LL', 'UL', 'UR', 'LR']
        points = [self.ll_point, self.ul_point, self.ur_point, self.lr_point]
        for label, pt in zip(labels, points):
            if pt is not None:
                cv2.putText(frame, label, (int(pt[0]) + 10, int(pt[1])), **debug_params)
        if self.ll_point is not None and self.lr_point is not None:
            lower_len = np.linalg.norm(self.lr_point - self.ll_point)
            cv2.putText(frame, f"Lower: {lower_len:.1f}px", (10, 100), **debug_params)
        if self.ul_point is not None and self.ur_point is not None:
            upper_len = np.linalg.norm(self.ur_point - self.ul_point)
            cv2.putText(frame, f"Upper: {upper_len:.1f}px", (10, 130), **debug_params)

    def draw_volume_table(self, frame: np.ndarray, volumes: list, table_x: int, table_y: int, track_id: int) -> None:
        """Draw a modern-looking table with diameter and volume data, including the track ID."""
        table_width = 250
        table_height = 150 + 50  # 5 rows: header + 4 diameters
        # Draw light gray background
        cv2.rectangle(frame, (table_x, table_y), (table_x + table_width, table_y + table_height), (220, 220, 220), -1)
        # Draw track ID above the table
        cv2.putText(frame, f"Syringe ID: {track_id}", (table_x + 10, table_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        # Draw headers
        cv2.putText(frame, "Diameter", (table_x + 10, table_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(frame, "mL", (table_x + 150, table_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        # Draw rows
        for i, (D, volume) in enumerate(volumes):
            y = table_y + 80 + i * 30
            cv2.putText(frame, f"{D:.2f}", (table_x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            if volume is not None:
                cv2.putText(frame, f"{volume:.2f}", (table_x + 150, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            else:
                cv2.putText(frame, "N/A", (table_x + 150, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        """Process a frame to detect multiple syringes, track them with ByteTrack, and display volume tables with IDs."""
        # Run YOLO model with ByteTrack tracking
        results = self.model.track(source=frame, persist=True, tracker='botsort.yaml', verbose=False, conf=0.4)
        if not results:
            return frame

        result = results[0]
        annotated_frame = result.plot()  # Draw bounding boxes with track IDs

        # Process each detected syringe
        for i, box in enumerate(result.boxes):
            # Extract track ID
            track_id = int(box.id) if box.id is not None else None
            if track_id is None:
                continue  # Skip if no track ID assigned

            # Extract bounding box coordinates
            box_coords = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = map(int, box_coords)

            # Verify keypoints availability
            if result.keypoints is None or len(result.keypoints.xy) <= i or len(result.keypoints.xy[i]) < 4:
                continue  # Skip if keypoints are insufficient

            # Extract keypoints
            try:
                kpts = result.keypoints.xy[i].cpu().numpy()
                ll_point, ul_point, ur_point, lr_point = kpts[:4]  # Lower-left, upper-left, upper-right, lower-right
            except Exception as e:
                print(f"Error extracting keypoints for syringe {track_id}: {e}")
                continue

            # Calculate volumes
            try:
                # Calculate edge lengths in pixels
                lower_edge_px = np.linalg.norm(lr_point - ll_point)
                upper_edge_px = np.linalg.norm(ur_point - ul_point)
                if lower_edge_px <= 0 or upper_edge_px <= 0:
                    continue
                avg_edge_px = (lower_edge_px + upper_edge_px) / 2.0

                # Calculate line equations
                lower_line_eq = self.calculate_line_equation(ll_point, lr_point)
                upper_line_eq = self.calculate_line_equation(ul_point, ur_point)

                # Calculate height in pixels
                h_pixels = self.calculate_parallel_distance(lower_line_eq, upper_line_eq)
                d_ll = self.point_to_line_distance(ll_point, upper_line_eq)
                d_lr = self.point_to_line_distance(lr_point, upper_line_eq)
                avg_point_dist = (d_ll + d_lr) / 2
                if h_pixels is not None and abs(h_pixels - avg_point_dist) <= 5:
                    height_pixels = h_pixels * 0.7 + avg_point_dist * 0.3
                else:
                    height_pixels = avg_point_dist

                # Calculate volumes for each diameter
                volumes = []
                for D in self.possible_diameters:
                    scale_factor_D = D / avg_edge_px
                    height_cm_D = height_pixels * scale_factor_D
                    if 0 < height_cm_D <= 30:  # Validate height
                        volume_D = math.pi * (D / 2) ** 2 * height_cm_D
                    else:
                        volume_D = None
                    volumes.append((D, volume_D))

                # Draw volume table with track ID
                table_x = x2 + 10  # Right of the bounding box
                table_y = y1       # Top of the bounding box
                self.draw_volume_table(annotated_frame, volumes, table_x, table_y, track_id)

            except Exception as e:
                print(f"Error processing syringe {track_id}: {e}")
                continue

        return annotated_frame

    def run(self) -> None:
        """Run the main loop to process the video file and save the output."""
        # Define input and output video paths
        input_path = "IMG_4938.mov"
        output_path = "processed_video2.mp4"

        # Open the input video
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise IOError(f"Cannot open video file: {input_path}")

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break  # End of video
                # Process the frame
                processed_frame = self.process_frame(frame)
                # Write the processed frame to the output video
                out.write(processed_frame)
                # Display the frame (optional, kept for monitoring)
                cv2.imshow('Syringe Volume Measurement', processed_frame)
                # Allow quitting by pressing 'q'
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        finally:
            # Clean up
            cap.release()
            out.release()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    calculator = SyringeVolumeCalculator()
    calculator.run()